In [2]:
import queue
import threading
import os
import requests
from urllib.request import urlopen, Request
from urllib.error import HTTPError
from bs4 import BeautifulSoup
import wget
from urllib import request
import urllib.request, urllib.error, urllib.parse
import re


/home/sleven/anaconda3/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [10]:
def mygobuster(turl):
    wordlist = [a.strip() for a in open("wordlist.txt")]
    filters = [".html",".css","jpg", "",".js",".md",".png"]
    for word in wordlist:
        for f in filters:
            url = turl+word+f
            re = requests.head(url)
            
            if re.status_code==301:
                print("Directory found: ", re.url, re.status_code)
                newurl = re.url+"/"

                mygobuster(newurl) 
            
            elif re.status_code==200 :
                print("Found: ", re.url, re.status_code)
                response = urllib.request.urlopen(url)
                if "png" in response or "jpg" in response:
                    Content = response.read()
                    f = open("found/"+str(re.url).replace("/",""), 'wb+')
                Content = response.read().decode('UTF-8')
                f = open("found/"+str(re.url).replace("/",""), 'w+')
                f.write(Content)
            
            else:
                continue
               
                

                

In [11]:
mygobuster("http://172.17.0.2:1337/")

Directory found:  http://172.17.0.2:1337/data 301
Directory found:  http://172.17.0.2:1337/data/about 301
Found:  http://172.17.0.2:1337/data/about/index.html 200
Directory found:  http://172.17.0.2:1337/data/faq 301
Found:  http://172.17.0.2:1337/data/faq/index.html 200
Directory found:  http://172.17.0.2:1337/data/impressum 301
Found:  http://172.17.0.2:1337/data/impressum/index.html 200
Directory found:  http://172.17.0.2:1337/data/datenschutz 301
Found:  http://172.17.0.2:1337/data/datenschutz/index.html 200
Directory found:  http://172.17.0.2:1337/data/about 301
Found:  http://172.17.0.2:1337/data/about/index.html 200
Directory found:  http://172.17.0.2:1337/data/impressum 301
Found:  http://172.17.0.2:1337/data/impressum/index.html 200
Directory found:  http://172.17.0.2:1337/img 301
Directory found:  http://172.17.0.2:1337/img/background-images 301
Directory found:  http://172.17.0.2:1337/img/FINDME 301
Found:  http://172.17.0.2:1337/about.html 200
Found:  http://172.17.0.2:1337

In [46]:
def threadbuster():
    threads = len(wordlist)
    for i in range(threads):
        t = threading.Thread(target=mygobuster)
        t.start()

In [12]:
import re
r  = requests.get(" http://172.17.0.2:1337/stylesheet.css")
data = r.text

l = []
for i in re.findall('url\(([^)]+)\)',data):
    l.append(i)
print(l)

['"/data/bgscreen.png"', '"/data/pexels-pixabay-273209.jpg"', '"img/background-images/pic2.jpg"']


In [13]:
t = "http://172.17.0.2:1337/"
s = [x[1:].strip('""') for x in l]
for i in s:
    print(t+i)

http://172.17.0.2:1337//data/bgscreen.png
http://172.17.0.2:1337//data/pexels-pixabay-273209.jpg
http://172.17.0.2:1337/img/background-images/pic2.jpg


In [74]:
class MySpider():
    def __init__(self):
        pass
        
    def mygobuster(self,target):
        wordlist = [a.strip() for a in open("wordlist.txt")]
        filters = [".html",".css","jpg", "",".js",".md",".png"]
        html = open("html.txt", "a+") 
        for word in wordlist:
            for f in filters:
                url = target+word+f
                re = requests.head(url)

                if re.status_code==301:
                    print("Directory found: ", re.url, re.status_code)
                    newurl = re.url+"/"
                    
                    self.mygobuster(newurl)

                elif re.status_code==200:
                    print("Found: ", re.url, re.status_code)
                    response = urllib.request.urlopen(url)
                    if ".html" in re.url:
                        html.write(re.url+"\n")
                    try:
                        Content = response.read().decode('utf-8')
                        f = open("found/"+str(re.url).replace("/",""), 'w+')
                        f.write(Content)
                        f.close()
                    except:
                        Content = response.read()
                        f = open("found/"+str(re.url).replace("/",""), 'wb+')
                        f.write(Content)
                        f.close()
                else:
                    continue

    def CSSbuster(self, target):
        r  = requests.get(" http://172.17.0.2:1337/stylesheet.css")
        data = r.text
        l = []
        for i in re.findall('url\(([^)]+)\)',data):
            l.append(i)
        s = [x[2:].strip('""') for x in l[:2]]
        for j in s:
            response = urllib.request.urlopen(target+j)
            if response.code==200:
                con = response.read()
                f = open("found/"+str(response.url).replace("/",""), 'wb+')
                f.write(con)
            else:
                pass
    
    
    def myBot():
    s=Service('/home/sleven/Studium/Sem5/Hacking-mit-Python/geckodriver')
    browser = webdriver.Firefox(service=s)
    for url in open("html.txt"):
        if url=="http://172.17.0.2:1337/biete.html\n":
            browser.get(url)
            eingabe = browser.find_element(By.ID,"ort")
            eingabe.send_keys("Salzburg")
            time.sleep(1)
            eingabe2 = browser.find_element(By.ID,"preis")
            eingabe2.send_keys("1500")
            time.sleep(1)
            eingabe2 = browser.find_element(By.ID,"zimmer")
            eingabe2.send_keys("5")
            time.sleep(1)
            eingabe3 = browser.find_element(By.ID,"flaeche")
            eingabe3.send_keys("500")
            time.sleep(1)
            eingabe4 = browser.find_element(By.ID,"kontakt")
            eingabe4.send_keys("Mori.r@web.de")
            time.sleep(1)

            ein = browser.find_element(By.ID,"saveimmoButton")
            ein.click()
            time.sleep(1)
        browser.get(url)
        time.sleep(1)


IndentationError: expected an indented block (<ipython-input-74-4825847e9af4>, line 56)

In [31]:
S = MySpider()

In [32]:
S.CSSbuster("http://172.17.0.2:1337/")

In [33]:
S.mygobuster("http://172.17.0.2:1337/")

Directory found:  http://172.17.0.2:1337/data 301
Directory found:  http://172.17.0.2:1337/data/about 301
Found:  http://172.17.0.2:1337/data/about/index.html 200
Directory found:  http://172.17.0.2:1337/data/faq 301
Found:  http://172.17.0.2:1337/data/faq/index.html 200
Directory found:  http://172.17.0.2:1337/data/impressum 301
Found:  http://172.17.0.2:1337/data/impressum/index.html 200
Directory found:  http://172.17.0.2:1337/data/datenschutz 301
Found:  http://172.17.0.2:1337/data/datenschutz/index.html 200
Directory found:  http://172.17.0.2:1337/data/about 301
Found:  http://172.17.0.2:1337/data/about/index.html 200
Directory found:  http://172.17.0.2:1337/data/impressum 301
Found:  http://172.17.0.2:1337/data/impressum/index.html 200
Directory found:  http://172.17.0.2:1337/img 301
Directory found:  http://172.17.0.2:1337/img/background-images 301
Found:  http://172.17.0.2:1337/about.html 200
Found:  http://172.17.0.2:1337/index.html 200
Found:  http://172.17.0.2:1337/biete.htm

**Selenium**

In [37]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import time
from selenium.webdriver.common.by import By

In [12]:
s=Service('/home/sleven/Studium/Sem5/Hacking-mit-Python/geckodriver')
browser = webdriver.Firefox(service=s)
url = 'http://172.17.0.2:1337/biete.html'
#driver = webdriver.Firefox(executable_path="/home/sleven/Studium/Sem5/Hacking-mit-Python/geckodriver")
browser.get(url)
time.sleep(2)

In [72]:
def myBot():
    s=Service('/home/sleven/Studium/Sem5/Hacking-mit-Python/geckodriver')
    browser = webdriver.Firefox(service=s)
    for url in open("html.txt"):
        if url=="http://172.17.0.2:1337/biete.html\n":
            browser.get(url)
            eingabe = browser.find_element(By.ID,"ort")
            eingabe.send_keys("Salzburg")
            time.sleep(1)
            eingabe2 = browser.find_element(By.ID,"preis")
            eingabe2.send_keys("1500")
            time.sleep(1)
            eingabe2 = browser.find_element(By.ID,"zimmer")
            eingabe2.send_keys("5")
            time.sleep(1)
            eingabe3 = browser.find_element(By.ID,"flaeche")
            eingabe3.send_keys("500")
            time.sleep(1)
            eingabe4 = browser.find_element(By.ID,"kontakt")
            eingabe4.send_keys("Mori.r@web.de")
            time.sleep(1)

            ein = browser.find_element(By.ID,"saveimmoButton")
            ein.click()
            time.sleep(1)
        browser.get(url)
        time.sleep(1)
        

In [73]:
myBot()

In [70]:
import urllib3

In [71]:
url = 'http://172.17.0.2:1337/'

conn = urllib3.
html = conn.read()

soup = BeautifulSoup(html)
links = soup.find_all('a')

for tag in links:
    link = tag.get('href',None)
    if link is not None:
        print(link)

AttributeError: module 'urllib3' has no attribute 'urlopen'